In [4]:
import pandas as pd
from math import ceil

In [5]:
# recipes are stored in dictionaires. Inputs and outputs are stored in lists of tuples.
# The first element of the tuple is the number of items needed/produced, the second element is the name
# of the item. The amount of time the recipe takes to make is given in seconds.
# assume 1 recipes per item
recipes = {'Iron Ingot': {'in': [(1,'Iron Ore')], 'out': [(1, 'Iron Ingot')], 'time': 1, 'machine': 'Smelter'},
           'Copper Ingot': {'in': [(1,'Copper Ore')], 'out': [(1, 'Copper Ingot')], 'time': 1, 'machine': 'Smelter'},
           'Stone Brick': {'in': [(1,'Stone')], 'out': [(1, 'Stone Brick')], 'time': 1, 'machine': 'Smelter'},
           'Energetic Graphite': {'in': [(2,'Coal')], 'out': [(1, 'Energetic Graphite')], 'time': 2, 'machine': 'Smelter'}, # has alternative recipes not included
           'High-Purity Silicon': {'in': [(2,'Silicon Ore')], 'out': [(1, 'High-Purity Silicon')], 'time': 2, 'machine': 'Smelter'},
           'Titanium Ingot': {'in': [(2,'Titanium Ore')], 'out': [(1, 'Titanium Ingot')], 'time': 2, 'machine': 'Smelter'},
           'Magnet': {'in': [(1,'Iron Ore')], 'out': [(1, 'Magnet')], 'time': 1.5, 'machine': 'Smelter'},
           'Glass': {'in': [(2,'Stone')], 'out': [(1, 'Glass')], 'time': 2, 'machine': 'Smelter'},
           'Diamond': {'in': [(1,'Kimberlite Ore')], 'out': [(2, 'Diamond')], 'time': 1.5, 'machine': 'Smelter'}, # has alternative recipes not included
           'Crystal Silicon': {'in': [(1,'Fractal Silicon')], 'out': [(2, 'Crystal Silicon')], 'time': 1.5, 'machine': 'Assembler'}, # has alternative recipes not included
           'Refined Oil': {'in': [(2,'Crude Oil')], 'out': [(2, 'Refined Oil'), (1, 'Hydrogen')], 'time': 4, 'machine': 'Refiner'}, # has alternative recipes not included

           'Steel': {'in': [(3,'Iron Ingot')], 'out': [(1, 'Steel')], 'time': 3, 'machine': 'Smelter'},
           'Gear': {'in': [(1,'Iron Ingot')], 'out': [(1, 'Gear')], 'time': 1, 'machine': 'Assembler'},
           'Prism': {'in': [(3,'Glass')], 'out': [(2, 'Prism')], 'time': 2, 'machine': 'Assembler'},
           'Circuit Board': {'in': [(2,'Iron Ingot'), (1,'Copper Ingot')], 'out': [(2, 'Circuit Board')], 'time': 1, 'machine': 'Assembler'},
           'Microcrystalline Component': {'in': [(2,'High-Purity Silicon'), (1,'Copper Ingot')], 'out': [(1, 'Microcrystalline Component')], 'time': 2, 'machine': 'Assembler'},
           'Magnetic Coil': {'in': [(2, 'Magnet'), (1,'Copper Ingot')], 'out': [(2, 'Magnetic Coil')], 'time': 1, 'machine': 'Assembler'},
           'Plastic': {'in': [(2, 'Refined Oil'), (1,'Energetic Graphite')], 'out': [(1, 'Plastic')], 'time': 3, 'machine': 'Chemical_Plant'},
           
           'Processor': {'in': [(2, 'Circuit Board'), (2, 'Microcrystalline Component')], 'out': [(1, 'Processor')], 'time': 3, 'machine': 'Assembler'},
           'Electric Motor': {'in': [(2,'Iron Ingot'), (1,'Gear'), (1,'Magnetic Coil')], 'out': [(1, 'Electric Motor')], 'time': 2, 'machine': 'Assembler'}, 
           'Titanium Alloy': {'in': [(4,'Titanium Ingot'), (4,'Steel'), (4,'Sulfuric Acid')], 'out': [(4, 'Titanium Alloy')], 'time': 12, 'machine': 'Smelter'}, 

           'Electromagnetic Turbine': {'in': [(2, 'Electric Motor'), (2, 'Magnetic Coil')], 'out': [(1, 'Electromagnetic Turbine')], 'time': 2, 'machine': 'Assembler'},

           'Super-Magnetic Ring': {'in': [(2, 'Electromagnetic Turbine'), (3, 'Magnet'), (1, 'Energetic Graphite')], 'out': [(1, 'Super-Magnetic Ring')], 'time': 3, 'machine': 'Assembler'},
          }
           

In [6]:
factory_props = {'Assembler': 1,
                 'Smelter': 1,
                 'Refiner': 1,
                 'Chemical_Plant': 1,
                 'Collider': 1,
                 'prolif': 1}

alternative_recipe_usage = {'Diamond': True}

def get_n(item_name, recipe):
    for out in recipe['out']:
        n, item = out
        if item == item_name:
            return n
    return None
    
def recursive_create_assembly_line(item_name, items_per_second, factory_props=factory_props, recipes=recipes):
    if item_name not in recipes.keys(): 
        return None

    # Get information about recipes and factory properties
    recipe = recipes[item_name]
    n = get_n(item_name, recipe)
    time = recipe['time']
    machine_rate = factory_props[recipe['machine']]
    prolif_rate = factory_props['prolif']

    # compute rates
    base_rate = n / time
    scaled_rate = base_rate * machine_rate * prolif_rate
    machines = items_per_second / scaled_rate

    # add information to dictionary
    assembly_line = {item_name: [machines, items_per_second]}

    # repeat process for all ingedients
    for ingedient in recipe['in']:
        n, sub_item = ingedient
        base_rate = (n / time) * machines
        sub_assembly_line = recursive_create_assembly_line(sub_item, base_rate, factory_props=factory_props, recipes=recipes)

        # update assembly_line dictionary
        if sub_assembly_line:
            for key, value in sub_assembly_line.items():
                if key in assembly_line.keys():
                    old_machines, old_ips = assembly_line[key]
                    sub_machines, sub_ips = value
                    assembly_line[key] = [old_machines + sub_machines, old_ips + sub_ips]
                else:
                    assembly_line[key] = value

    return assembly_line

def create_assembly_line(item_name, items_per_second, factory_props=factory_props, recipes=recipes):
    assembly_line = recursive_create_assembly_line(item_name, items_per_second, factory_props=factory_props, recipes=recipes)
    return pd.DataFrame.from_dict(assembly_line, orient ='index', columns=['Machines','Items/Second'])

In [7]:
recursive_create_assembly_line('Processor', 1, factory_props=factory_props)

{'Processor': [3.0, 1],
 'Circuit Board': [1.0, 2.0],
 'Iron Ingot': [2.0, 2.0],
 'Copper Ingot': [3.0, 3.0],
 'Microcrystalline Component': [4.0, 2.0],
 'High-Purity Silicon': [8.0, 4.0]}

In [8]:
recursive_create_assembly_line('Electric Motor', 1, factory_props=factory_props)

{'Electric Motor': [2.0, 1],
 'Iron Ingot': [3.0, 3.0],
 'Gear': [1.0, 1.0],
 'Magnetic Coil': [0.5, 1.0],
 'Magnet': [1.5, 1.0],
 'Copper Ingot': [0.5, 0.5]}

In [9]:
create_assembly_line('Electromagnetic Turbine', 1, factory_props=factory_props)

,Machines,Items/Second
Electromagnetic Turbine,2.0,1.0
Electric Motor,4.0,2.0
Iron Ingot,6.0,6.0
Gear,2.0,2.0
Magnetic Coil,2.0,4.0
Magnet,6.0,4.0
Copper Ingot,2.0,2.0


In [10]:
create_assembly_line('Super-Magnetic Ring', 1, factory_props=factory_props)

,Machines,Items/Second
Super-Magnetic Ring,3.0,1.0
Electromagnetic Turbine,4.0,2.0
Electric Motor,8.0,4.0
Iron Ingot,12.0,12.0
Gear,4.0,4.0
Magnetic Coil,4.0,8.0
Magnet,16.5,11.0
Copper Ingot,4.0,4.0
Energetic Graphite,2.0,1.0
